In [186]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split #generar set de prueba
from sklearn.linear_model import LogisticRegression #regresión logística
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix, roc_curve, roc_auc_score
import matplotlib.pyplot as plt


In [187]:
df = pd.read_excel("df_limpio.xlsx")
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 22735 entries, 0 to 22734
Data columns (total 39 columns):
 #   Column              Non-Null Count  Dtype         
---  ------              --------------  -----         
 0   folio               22735 non-null  int64         
 1   tag                 22735 non-null  object        
 2   folio_solicitud     22735 non-null  int64         
 3   fecha               22735 non-null  datetime64[ns]
 4   marca               22735 non-null  object        
 5   modelo              22735 non-null  object        
 6   plazo               22735 non-null  int64         
 7   precio              22735 non-null  float64       
 8   enganche            22735 non-null  float64       
 9   descuento           22735 non-null  float64       
 10  semana              22735 non-null  int64         
 11  monto_financiado    22735 non-null  float64       
 12  costo_total         22735 non-null  int64         
 13  monto_accesorios    22735 non-null  float64   

In [188]:
#plazo - precio - enganche - semana - montofinanciado - costototal - pagosrealizados - statuscuenta - porceng - limite crédito (porcentaje financiado vs el otorgado) - semanaActual -
Ev = df.iloc[ : , [6,7,8,10,11,12,20,25,26,27,29,30,31,33]]
Ev.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 22735 entries, 0 to 22734
Data columns (total 14 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   plazo             22735 non-null  int64  
 1   precio            22735 non-null  float64
 2   enganche          22735 non-null  float64
 3   semana            22735 non-null  int64  
 4   monto_financiado  22735 non-null  float64
 5   costo_total       22735 non-null  int64  
 6   pagos_realizados  22735 non-null  int64  
 7   puntos            22735 non-null  int64  
 8   riesgo            22735 non-null  float64
 9   score_buro        22735 non-null  int64  
 10  porc_eng          22735 non-null  float64
 11  limite_credito    22735 non-null  int64  
 12  semana_actual     22735 non-null  int64  
 13  edad_cliente      9627 non-null   float64
dtypes: float64(6), int64(8)
memory usage: 2.4 MB


In [189]:
Ev = Ev[(Ev["limite_credito"]>0)]
Ev.info()

<class 'pandas.core.frame.DataFrame'>
Index: 22614 entries, 0 to 22734
Data columns (total 14 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   plazo             22614 non-null  int64  
 1   precio            22614 non-null  float64
 2   enganche          22614 non-null  float64
 3   semana            22614 non-null  int64  
 4   monto_financiado  22614 non-null  float64
 5   costo_total       22614 non-null  int64  
 6   pagos_realizados  22614 non-null  int64  
 7   puntos            22614 non-null  int64  
 8   riesgo            22614 non-null  float64
 9   score_buro        22614 non-null  int64  
 10  porc_eng          22614 non-null  float64
 11  limite_credito    22614 non-null  int64  
 12  semana_actual     22614 non-null  int64  
 13  edad_cliente      9585 non-null   float64
dtypes: float64(6), int64(8)
memory usage: 2.6 MB


In [190]:
#Encontramos el punto de equilibrio
Ev["PE"] = Ev["semana"]*Ev["pagos_realizados"]
#Porcentaje monto financiado
Ev["porcFinan"] = (Ev["monto_financiado"]/Ev["limite_credito"])*100

In [191]:


# Aplicar una condición if para comparar las dos columnas y obtener un resultado
def comparar_pagos(row):
    if row['monto_financiado'] > row['PE']:
        return "NoCompletóPago"
    elif row['PE'] > row['monto_financiado']:
        return "CompletóPago"
    else:
        return "Igual"

# Aplicar la función y crear una nueva columna con los resultados
Ev['VR'] = Ev.apply(comparar_pagos, axis=1)

# Mostrar el DataFrame resultante
print(Ev)


       plazo  precio  enganche  semana  monto_financiado  costo_total  \
0         26  1949.0     780.0      81            1169.0         2106   
1         13  2999.0    1050.0     225            1949.0         2925   
3         13  1959.0     490.0     170            1469.0         2210   
4         39  4299.0    1075.0     178            3224.0         6942   
5         26  2999.0     750.0     156            2249.0         4056   
...      ...     ...       ...     ...               ...          ...   
22730     26  3349.0     297.0     211            3052.0         5486   
22731     26  4100.0    1209.0     200            2891.0         5200   
22732     26  4849.0    1152.0     256            3697.0         6656   
22733     39  9799.0    2799.0     386            7000.0        15054   
22734     26  5399.0    1086.0     299            4313.0         7774   

       pagos_realizados  puntos  riesgo  score_buro  porc_eng  limite_credito  \
0                     0       0    0.00   

In [192]:
frecuenciaVR = Ev["VR"].value_counts()
frecuenciaVR

VR
NoCompletóPago    14556
CompletóPago       8058
Name: count, dtype: int64

In [193]:
X = Ev[["porcFinan","score_buro","plazo","precio","enganche","puntos","monto_financiado","costo_total"]]
y = Ev[["VR"]]

In [194]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3)


In [195]:
logistic_model = LogisticRegression()
logistic_model.fit(X_train, y_train)

c:\Users\Carlos Casillas\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\utils\validation.py:1184: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


LogisticRegression()

In [196]:
y_pred = logistic_model.predict(X_test)


In [197]:
accuracy = accuracy_score(y_test, y_pred)
print(f'Accuracy: {accuracy:.2f}')

Accuracy: 0.68


In [198]:
confusion = confusion_matrix(y_test, y_pred)
print('Confusion Matrix:')
print(confusion)

Confusion Matrix:
[[ 590 1789]
 [ 402 4004]]


In [199]:
class_report = classification_report(y_test, y_pred)
print('Classification Report:')
print(class_report)

Classification Report:
                precision    recall  f1-score   support

  CompletóPago       0.59      0.25      0.35      2379
NoCompletóPago       0.69      0.91      0.79      4406

      accuracy                           0.68      6785
     macro avg       0.64      0.58      0.57      6785
  weighted avg       0.66      0.68      0.63      6785

